# MNIST data classification: Keras and input pipeline

This notebook shows how to use input pipelines with Keras.

In [1]:
import numpy as np
import math
import tensorflow as tf

In [2]:
tf.version.VERSION

'2.3.0'

In [3]:
def decode(serialized_example):
    """Parses an image and label from the given `serialized_example`."""
    features = tf.io.parse_single_example(
        serialized_example,
        features={
            'image_raw': tf.io.FixedLenFeature([], tf.string),
            'label': tf.io.FixedLenFeature([], tf.int64),
        })
    label = tf.cast(features['label'], tf.int32)
    image = tf.io.decode_raw(features['image_raw'], tf.uint8)
    return image, label


def normalize(image, label):
    """Convert `image` from [0, 255] -> [-0.5, 0.5] floats."""
    image = tf.cast(image, tf.float32) * (1. / 255) - 0.5
    return image, label

In [26]:
dataset = tf.data.TFRecordDataset('../tfrecords/train.tfrecords')
dataset = dataset.map(decode)
dataset = dataset.map(normalize)
dataset = dataset.shuffle(6000)
dataset = dataset.batch(256)
dataset = dataset.repeat(5)

In [27]:
test_set = tf.data.TFRecordDataset('../tfrecords/test.tfrecords')
test_set = test_set.map(decode)
test_set = test_set.map(normalize)
test_set = test_set.batch(256)

In [28]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(784,)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [29]:
opt = tf.keras.optimizers.Adam(0.001)

In [30]:
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [31]:
fit = model.fit(dataset,
                epochs=5,
                steps_per_epoch=math.ceil(60000 / 256), validation_data=test_set)

Epoch 1/5
235/235 [==============================] - 3s 14ms/step - loss: 0.4984 - accuracy: 0.8570 - val_loss: 0.2871 - val_accuracy: 0.9181
Epoch 2/5
235/235 [==============================] - 3s 12ms/step - loss: 0.2652 - accuracy: 0.9235 - val_loss: 0.2223 - val_accuracy: 0.9358
Epoch 3/5
235/235 [==============================] - 3s 12ms/step - loss: 0.2082 - accuracy: 0.9400 - val_loss: 0.1849 - val_accuracy: 0.9455
Epoch 4/5
235/235 [==============================] - 3s 12ms/step - loss: 0.1694 - accuracy: 0.9509 - val_loss: 0.1571 - val_accuracy: 0.9535
Epoch 5/5
235/235 [==============================] - 3s 12ms/step - loss: 0.1429 - accuracy: 0.9585 - val_loss: 0.1427 - val_accuracy: 0.9567
